# **Jupyter Notebook for GCR**
</br>

## **Table of Content**

>### 0. Introduction
>### 1. 환경 구성
>### 2. Train Workflow
>### 3. Inference Workflow
>### 4. Batch Running
>### 5. 문의 및 기능 개발 요청
>### 6. References   

<br>

## **0. Introduction**

#### 본 sample notebook은 GCR의 구조와 각 asset의 역할과 산출물, 그 사용법을 처음 접하는 분들이 알기 쉽게 이해할 수 있도록 제작되었습니다.   

[ **1. 환경 구성** ]에서는 ALO 등 환경 설치 방법을 설명하고,   
[ **2. Train workflow** ]와 [ **3. Inference workflow** ]는 각각 train workflow와 inference workflow의 사용 방법과 산출물을 설명하며,   
[ **4. Batch running** ]에서는 sample notebook이 아닌 실제 과제 운용 시에 GCR contents를 수행하는 방법을 설명하고   
[ **5. 문의 및 기능 개발 요청** ]에서는 사용 중 문의 사항이나 기능에 대한 수정/개발 요청 방법을,   
[ **6. References** ]에서는 추가로 참고하실 collab 문서 등에 대한 links를 제공합니다.

#### Notebook의 Workflow는 다음과 같이 구성됩니다.
> Workflow NAME (ex. Train Workflow)

>> Workflow 구성 설명
>>> **A** asset : A asset 설명   
>>> **B** asset : B asset 설명   
>>> ...

>> Workflow Setup 
>>> Workflow Setup 코드

>> [0] **A** asset
>>> parameter 설명 및 실행 코드 

>> [1] **B** asset
>>> parameter 설명 및 실행 코드

>> ..

</br>

#### 각 Asset은 동작확인을 위해 다음과 같이 구성됩니다.
> ASSET NAME

>> 주요 Parameter 설명
>>> param1: param1 설명   
>>> ***param2***: param2 설명 [*option1 / option2 / option3*]   
>>> param3: param3 설명 [*option1 / option2*]   
>>> ..

>> Parameter 설정부
>>> #################   
>>> parameter 설정 코드   
>>> #################   

>> Asset 실행부
>>> #################   
>>> Asset 실행 코드   
>>> #################   

Asset 별로 experimental_plan.yaml에 주어진 parameter에 대한 설명이 주어집니다.   
설명에 따라 parameter 변경 시 experimental_plan.yaml을 직접 수정하거나, Parameter 설정부에서 코드 실행을 통해 바꿀 수 있습니다.   
위에서 param2와 같이 ***Bold Italic***으로 표기된 변수는 필수 설정 변수로, 최초 실행 또는 데이터가 바뀔 시 꼭 다시 설정해주어야 하는 변수를 의미합니다.   
설정된 parameter로 asset을 실행할 수 있습니다. 또한 Parameter를 변경해가며 Asset 실행 결과 변화를 관찰할 수 있습니다.

</br>

## **1. 환경 구성**

#### GCR을 사용하기 위해서는 아래와 같은 방법으로 데이터를 준비해야 합니다.
> 1. Train, Inference 두 개의 데이터셋을 준비합니다.
> 2. 각 데이터에 FLAG_TRAIN_INFERENCE 컬럼을 추가합니다. 
>    - 각 'Train', 'Inference'가 flag로 들어가야 합니다.
> 3. 두 데이터를 합쳐 하나의 데이터셋으로 구성합니다.

***GCR은 결측치와 범주형 데이터에 대한 전처리가 필요하지 않습니다***

#### ALO 설치 및 configuration 설정 방법은 다음과 같습니다.

1. 최상위 디렉토리에서 install.sh를 실행합니다
> source install.sh
2. install.sh를 실행하면 alo 디렉토리가 설치됩니다.
3. 가상환경을 설치 및 실행합니다.
> conda create -n gcr python=3.10   
> conda init bash   
> conda activate gcr    
3. alo/config 디렉토리로 이동하여 experimental_plan.yaml 파일을 오픈합니다.
4. external_path의 load_train_data_path에 아래와 같이 사용할 데이터의 경로(디렉토리)를 입력합니다.

>```
>external_path:
>    - load_train_data_path: /nas001/gcr_test_data/sample/
>    - load_inference_data_path:
>    - save_train_artifacts_path:
>    - save_inference_artifacts_path:
>```

5. 필수 변경 parameter를 변경합니다. 나머지 parameter는 컨텐츠 yaml에 제공된 default 값을 사용해도 괜찮습니다.
6. 아래 **ALO Setup**을 실행합니다.

### ALO Setup
라이브러리 설치 및 컨텐츠 다운로드를 위해 아래 코드를 실행 해주세요

In [ ]:
import argparse
import time
import os
import copy
os.chdir(os.path.abspath(os.path.join('./alo')))

from src.alo import ALO
from src.alo import AssetStructure
alo = ALO(); alo.set_proc_logger(); alo.preset()
pipelines = list(alo.asset_source.keys())

from src.external import external_load_data, external_save_artifacts
def run(step, pipeline, asset_structure):
    # 반복되는 작업을 함수로 변환
    asset_config = alo.asset_source[pipeline]
    return alo.process_asset_step(asset_config[step], step, pipeline, asset_structure)

# pipeline list 를 가지고 옴
pipelines = list(alo.asset_source.keys())
pipelines

## **2. Train Workflow**

#### GCR의 Train Workflow 구성은 다음과 같습니다.
> **[0]** Input asset : *사용자가 지정한 경로로부터 데이터를 Import*   
> **[1]** Graph asset : *데이터를 토대로 그래프를 구성하고 필요한 임베딩 추출*   
> **[2]** Preprocess asset : *(필요시) 결측치 처리 및 라벨 인코딩*   
> **[3]** Sampling asset : *(필요시) Imbalance 데이터의 Undersampling*   
> **[4]** Train asset : *ML 모델 학습*

#### Train Workflow Setup
아래 코드를 실행하여 Train Workflow에 필요한 라이브러리를 먼저 설치 해주세요.

In [ ]:
alo.external_load_data(pipelines[0]) # external load data for train_pipeline
# 사용하는 pipeline의 package를 설치
# train = 0, infernence = 1을 선택해야 하고 둘다 설치 해야함
pipeline = pipelines[0]
alo.install_steps(pipeline, alo.control["get_asset_source"])
 # 초기 data structure 구성
alo.set_asset_structure()
init_asset_structure = copy.deepcopy(alo.asset_structure)

### [0] Input asset

#### 주요 Parameter
- ***input_path*** : Extenal Path에서 받은 데이터는 *alo/input/train*에 저장됩니다. 이 중에서 사용할 데이터가 저장된 디렉터리 명을 작성해주시면 됩니다.
- x_columns : 데이터의 모든 컬럼을 활용하지 않는 경우엔 직접 선택해서 사용할 수 있습니다.
- use_all_x : 데이터의 모든 컬럼을 사용하는 경우 True로 설정하고 사용합니다. 이 경우 x_columns는 빈칸이어야 합니다. [*True / False*]
- ***y_column*** : Classification, Regression을 위해서는 Label이 있어야 합니다. Label에 해당하는 컬럼을 작성합니다.
- groupkey_columns : 특정 컬럼 명을 기준으로 데이터를 그룹으로 나누어 모델링을 하고 싶을 경우에 사용합니다.
- drop_columns : use_all_x가 True일 때 삭제하고 싶은 컬럼을 입력합니다.
- time_column : 데이터에 시간 컬럼이 있을 경우 입력합니다.
- concat_dataframes : 같은 형태 csv 파일 여러 개를 input data로 불러올 시, concat 여부를 선택합니다. [*True / False*]
- encoding : pd.read_csv() 시에 사용할 encoding 방법을 설정합니다.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - graph(1) - preprocess(2) - sampling(3) - train(4))
step = 0
alo.asset_structure= copy.deepcopy(init_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 input asset argument를 원하는 값으로 수정합니다. 
#asset_structure.args['x_columns'] = ['']
alo.asset_structure.args

#### Input asset 실행 

In [ ]:
input_asset_structure = run(step, pipeline, alo.asset_structure)

# input asset의 결과 dataframe은 input_asset_structure.data['dataframe']으로 확인할 수 있습니다.
input_asset_structure.data['dataframe'].head(10)

### [1] Graph asset

GCR의 가장 핵심이 되는 asset입니다. Raw Data를 임베딩으로 모두 변환하여 별도의 전처리 없이 더 높은 성능의 ML모델을 획득하기 위한 Tool이 됩니다.
#### 주요 Parameter
- graph_type : 그래프의 구조를 선택할 수 있습니다. radial은 방사형 구조, relational은 관계형 구조입니다. [*radial / relational*]
- center_node_column : 방사형 그래프는 중심 컬럼을 기준으로 그래프를 구성합니다. 데이터의 대표가 되는 컬럼을 선택하면 됩니다.(ex. ID, Name 등)
- embedding_column : 임베딩의 대상이 되는 컬럼을 선택합니다. 라벨이 center_node_column과 대응하지 않는 경우가 있을 수 있습니다. 이때는 label이 대응하는 컬럼을 embedding column으로 설정합니다.
- drop_columns : 그래프 구성에서 제외할 컬럼을 선택합니다.
- num_epochs: Embedding 학습을 위한 epoch을 설정합니다.
- workers : worker process 수를 설정합니다.
- num_partitions : partition 수를 설정합니다. 증가시켜 메모리 사용량을 줄일 수 있습니다.
- extra_columns_for_ml : ml 모델 학습에 임베딩 외에 사용될 컬럼을 지정합니다. 도메인 지식에 기반하여 선택할 필요가 있습니다.
- custom_connection : 그래프를 활용할 줄 안다면 직접 relation을 정의할 수 있습니다. 자세한 내용은 manual을 참고.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - graph(1) - preprocess(2) - sampling(3) - train(4))
step = 1 
alo.asset_structure= copy.deepcopy(input_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 graph asset argument를 원하는 값으로 수정합니다. 
#asset_structure.args['dimension'] = 128
alo.asset_structure.args

#### Graph asset 실행 

In [ ]:
# asset 실행
graph_asset_structure=run(step, pipeline, alo.asset_structure)

# graph asset의 결과 dataframe은 graph_asset_structure.data['dataframe']으로 확인할 수 있습니다. 
graph_asset_structure.data['dataframe'].head(10)

### [2] Preprocess asset 

GCR은 데이터 전처리가 불필요하기 때문에 Preprocess asset의 역할은 크지 않습니다. 다만 사용자가 임베딩 외에 raw data를 학습에 사용하는 경우 (즉, extra_columns_for_ml 설정시) 결측치를 처리하기 위한 용도입니다.
#### 주요 Parameter
- handling_missing : 결측치 처리 방식을 지정합니다. 'interpolation' 또는 'fill_number' 중에 선택할 수 있으며 GCR에서는 'interpolation'을 권장합니다.
- ***handling_encoding_y_column*** : input asset의 y_column과 동일하게 설정합니다. (필수)
- ***handling_encoding_y*** : y_column의 인코딩 방식을 설정합니다. GCR에서는 'label'로 설정합니다.
- handling_scaling_x : X 컬럼의 scaling 방식을 선택합니다.
- load_train_preprocess : False로 설정합니다. (inference workflow 전용)

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - graph(1) - preprocess(2) - sampling(3) - train(4))
step = 2 
alo.asset_structure = copy.deepcopy(graph_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 preprocess asset argument를 원하는 값으로 수정합니다. 
# asset_structure.args['handling_missing'] = dropna
alo.asset_structure.args

#### Preprocess asset 실행 

In [ ]:
# asset 실행
preprocess_asset_structure=run(step, pipeline, alo.asset_structure)

# preprocess asset의 결과 dataframe은 preprocess_asset_structure.data['dataframe']으로 확인할 수 있습니다.  
preprocess_asset_structure.data['dataframe'].head(10)

### [3] Sampling asset  

데이터 imbalance가 심한경우엔 sampling이 필요할 수 있습니다. 현재는 undersampling만 제공됩니다.
#### 주요 Parameter
- sampling_type : sampling이 필요한 경우 설정합니다. 필요없는 경우엔 'none'으로 설정합니다. [*none / under*]
- sampling_method : sampling 방법을 선택합니다. [*random / cluster / negative*]
- label_sampling : 데이터의 Y 라벨을 기준으로 데이터를 그룹핑하여 샘플링을 할 지 선택합니다. [*True / False*]
- ignore_label_class : label_sampling이 True일 때, 라벨의 특정 클래스는 샘플링하지 않고 전부 사용하고 싶을 때 사용합니다. EX) Y 라벨 클래스가 'OK' 또는 'NG'일 때 'NG'클래스를 입력할 경우 NG클래스는 샘플링하지 않습니다.
- negative_target_class : Negative sampling(sampling_method가 negative일 때)에만 사용되는 argument입니다. 기준이 되는 Y 라벨의 특정 클래스를 기입합니다.
- label_sampling_num_type : Y 라벨의 각 클래스에 샘플링할 데이터 수를 계산하기 위한 방법들입니다. [*ratio / number / compare / mingroup*]
- label_sampling_num : label_sampilng_num_type에 따라 값을 입력합니다. (user manual 참조)
- sampling_groupkey_columns : 특정 컬럼을 기준으로 데이터를 그룹핑하여 샘플링을 할 때 사용합니다. 컬럼명을 list로 입력합니다.
- sampling_num_type : 샘플링할 데이터 수를 결정하는 방법입니다. [*ratio / number / compare / mingroup*]
- sampling_num : sampling_num_type에 따라 아래 예시의 형식으로 값을 입력합니다. (user manual 참조)

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - graph(1) - preprocess(2) - sampling(3) - train(4))
step = 3 
alo.asset_structure = copy.deepcopy(preprocess_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 sampling asset argument를 원하는 값으로 수정합니다. 
# asset_structure.args['sampling_type'] = 'under'
alo.asset_structure.args

#### Sampling asset 실행 

In [ ]:
# asset 실행
sampling_asset_structure=run(step, pipeline, alo.asset_structure)

# sampling asset의 결과 dataframe은 sampling_asset_structure.data['dataframe']으로 확인할 수 있습니다.
sampling_asset_structure.data['dataframe'].head(10)

### [4] Train asset

추출된 임베딩을 활용하여 ML모델을 학습합니다.
#### 주요 Parameter
- model_type: 목적에 맞는 학습 방식을 선택합니다. (classification/regression)
- data_split_method: HPO를 위한 데이터 분할 방식을 선택합니다. (cross_validate/train_test_split)
- evaluation_metric: classification의 경우 accuracy, precision, recall, f1-score / regression의 경우 mse, r2, mae, rmse 중 선택합니다.
- model_list: lightgbm, random-forest, gbm, Catboost 중 복수 선택 가능합니다.
- num_hpo: 설정 범위 내 hpo 횟수를 결정합니다.
- param_range: Search 범위를 지정합니다.
- shap_ratio: shap value 뽑을 데이터를 sampling 하는 비율을 결정합니다.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - graph(1) - preprocess(2) - sampling(3) - train(4))
step = 4 
alo.asset_structure = copy.deepcopy(sampling_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 train asset argument를 원하는 값으로 수정합니다.
alo.asset_structure.args['model_list'] = ['lgb']
alo.asset_structure.args

#### Train asset 실행 

In [ ]:
# asset 실행
train_asset_structure=run(step, pipeline, alo.asset_structure)

# train asset의 결과 dataframe은 train_asset_structure.data['dataframe']으로 확인할 수 있습니다.
train_asset_structure.data['dataframe'].head(10)

## **3. Inference Workflow**

#### GCR의 Inference Workflow 구성은 다음과 같습니다.
> **[0]** Input : *사용자가 지정한 경로로부터 데이터를 Import*   
> **[1]** Preprocess : *(필요시) 결측치 처리 및 라벨 인코딩*   
> **[2]** Inference : *Train Workflow에서 선택된 베스트 모델을 활용해 라벨 추론*   
> **[3]** Result : *결과 출력*   


#### Inference Workflow Setup
아래 코드를 실행하여 Inference Workflow에 필요한 라이브러리를 먼저 설치 해주세요.

In [ ]:
# 아래는 Inference 시 필요한 라이브러리를 설치하는 코드입니다. library 설치 에러가 발생하면 아래 셀을 재실행 해주세요
alo.external_load_data(pipelines[1]) # external load data for train_pipeline
# 사용하는 pipeline의 package를 설치
# train = 0, infernence = 1을 선택해야 하고 둘다 설치 해야함
pipeline = pipelines[1]
alo.install_steps(pipeline, alo.control["get_asset_source"])
 # 초기 data structure 구성
alo.set_asset_structure()
init_asset_structure = copy.deepcopy(alo.asset_structure)

### [0] Input asset 

#### 주요 Parameter
- input_path : GCR에서는 추론데이터가 'inference' 위치에 자동 저장됩니다. 따로 설정할 필요 없이 주어진 'inference'로 놓고 사용합니다.
- x_columns : Inference workflow에서는 x_column을 따로 지정하지 않습니다. None으로 설정합니다.
- use_all_x : Inference workflow에서는 use_all_x를 True로 놓습니다.
- y_column : 추론데이터는 y_column이 없습니다. None으로 설정합니다.
- groupkey_columns : 특정 컬럼 명을 기준으로 데이터를 그룹으로 나누어 모델링을 하고 싶을 경우에 사용합니다.
- drop_columns : use_all_x가 True일 때 삭제하고 싶은 컬럼을 입력합니다.
- time_column : 데이터에 시간 컬럼이 있을 경우 입력합니다.
- concat_dataframes : 같은 형태 csv 파일 여러 개를 input data로 불러올 시, concat 여부를 선택합니다. [*True / False*]
- encoding : pd.read_csv() 시에 사용할 encoding 방법을 설정합니다.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2) - result(3))
step = 0 
alo.asset_structure = copy.deepcopy(init_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 input asset argument를 원하는 값으로 수정합니다.
# asset_structure.args['x_columns'] = ['']
alo.asset_structure.args

##### Input asset 실행 

In [ ]:
# asset 실행
input_asset_structure=run(step, pipeline, alo.asset_structure)

# input asset의 결과 dataframe은 input_asset_structure.data['dataframe']으로 확인할 수 있습니다.
input_asset_structure.data['dataframe'].head(10)

### [1] Preprocess asset 

GCR은 데이터 전처리가 불필요하기 때문에 Preprocess asset의 역할은 크지 않습니다. 다만 사용자가 임베딩 외에 raw data를 학습에 사용하는 경우 (즉, extra_columns_for_ml 설정시) 결측치를 처리하기 위한 용도입니다.
#### 주요 Parameter
- handling_missing: 결측치 처리 방식을 지정합니다. 'interpolation' 또는 'fill_number' 중에 선택할 수 있으며 GCR에서는 'interpolation'을 권장합니다.
- ***handling_encoding_y_column***: None으로 설정합니다.
- limit_encoding_categories: onehot이나 hashing 인코딩 진행 시 컬럼이 너무 많아지는 것에 대한 한계치를 설정합니다.
- load_train_preprocess: 반드시 True로 설정합니다. train workflow의 preprocess를 참조하여 진행합니다.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2) - result(3))
step = 1 
alo.asset_structure = copy.deepcopy(input_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 preprocess asset argument를 원하는 값으로 수정합니다.
# asset_structure.args['x_columns'] = ['']
alo.asset_structure.args

#### Preprocess asset 실행 

In [ ]:
# asset 실행
preprocess_asset_structure=run(step, pipeline, alo.asset_structure)

# preprocess asset의 결과 dataframe은 preprocess_asset_structure.data['dataframe']으로 확인할 수 있습니다.  
preprocess_asset_structure.data['dataframe'].head(10)

### [2] Inference asset 

#### 주요 Parameter
- model_type: Train workflow의 Train asset과 동일하게 classification/regression 중 설정하면 됩니다. [*classification / regression*]
- run_shapley: shapley 실행 여부를 선택합니다. [*True / False*]

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2) - result(3))
step = 2 
alo.asset_structure = copy.deepcopy(preprocess_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 inference asset argument를 원하는 값으로 수정합니다.
# asset_structure.args['x_columns'] = ['']
alo.asset_structure.args

#### Inference asset 실행 

In [ ]:
# asset 실행
inference_asset_structure=run(step, pipeline, alo.asset_structure)

# inference asset의 결과 dataframe은 inference_asset_structure.data['dataframe']으로 확인할 수 있습니다.  
inference_asset_structure.data['dataframe'].head(10)

### [3] Result asset

#### 주요 Parameter
- result_save_name: 결과 저장 파일명을 설정합니다.

#### Parameter 설정

In [ ]:
# GCR asset 순서에 따라 step 순서를 입력합니다. (input(0) - preprocess(1) - inference(2) - result(3))
step = 3
alo.asset_structure = copy.deepcopy(inference_asset_structure)
alo.asset_structure.args = alo.get_args(pipeline, step)

# 아래 주석을 풀어 result asset argument를 원하는 값으로 수정합니다.
# asset_structure.args['x_columns'] = ['']
alo.asset_structure.args

#### Result asset 실행 

In [ ]:
# asset 실행
result_asset_structure=run(step, pipeline, alo.asset_structure)

# result asset의 결과 dataframe은 result_asset_structure.data['dataframe']으로 확인할 수 있습니다.
result_asset_structure.data['dataframe'].head(10)

## **4. Batch Running**

Asset 단위가 아닌 전체 workflows에 대해 한번에 동작 시킬 수 있습니다.
> 1. alo 디렉토리로 이동합니다.    
>> cd alo    
> 2. main.py를 실행합니다. 
>> python main.py

*Sample Notebook에서 반영한 parameter는 experimental_plan.yaml에 반영되지 않습니다.*   
*config/experimental_plan.yaml을 직접 수정하여 사용하시길 바랍니다.*

## **5. 문의 및 기능 개발 요청**

사용중 **Issue 발생** 또는 **기능 요청** 건이 있으실 경우 아래 CLM을 통해 문의/요청 바랍니다.   
CLM : http://clm.lge.com/issue/projects/AICONTENTS

담당자: 공성우 선임, 김정원 연구원

*긴급한 건에 대해서는 담당자에게 연락 바랍니다.*

## **6. References**

GCR Release Note : http://collab.lge.com/main/x/iIjdgQ

User Guide : http://collab.lge.com/main/x/Owo8gg

데이터 명세서 : http://collab.lge.com/main/x/QAo8gg

알고리즘 설명서 : http://collab.lge.com/main/x/Zgo8gg

GCR Contents Git : http://mod.lge.com/hub/dxadvtech/aicontents/gcr